# 分类

In [1]:
# KNN 
# 朴素贝叶斯
# 决策树
# 支持向量机
# 集成方法
# 罗吉斯特映射（以回归为主，可做分类，可做回归）
# 人工神经网络（以回归为主，可做分类，可做回归）

In [2]:
# 距离
# 欧式距离：两个点的直线距离
# 曼哈顿距离：各个维度的长度差的绝对值的和（横坐标的长度差加上纵坐标的长度差）
# 闵可夫斯基距离：各个维度的长度差的绝对值p次方的和，开p次根号
# KD-Tree
# 如果一个空间中有很多的点，那么如何找到我们随机指定的一个点的附近的K个点。
# 通过一个维度将所有的点分成两个部分，两部分的数量尽可能的保持一致。
# 然后拿其中的一部分再进行另一个维度上的切分，一次类推，直到不能切分了为止
# 这样的话，在空间中就建立了许多大小不一的格子，每个点通过格子的线进行链接（线当作是中间节点，点当作叶子节点），构成了一个树形结构。

### KNN （K-Nearest Neighbors）

In [3]:
# 算法思想
# 一个数据集都会有它的标注，那么如果找到了一个点，它k个最近的邻居，一种标注大于了另外一种标注，那么我们就认为，这个点更倾向于与多数点是一致的。

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [29]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)
        
    
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    # print(len(X_train), len(X_validation), len(X_test))
    
    # 衡量指标
    # accuracy_score 准确率
    # recall_score 召回率
    # f1_score F值
    from sklearn.metrics import accuracy_score, recall_score, f1_score
    # KNN
    # NearestNeighbors 此方法可以直接获得一个点附近最近的几个点
    from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
    models = []
    # n_neigbors 获取最近的几个点
    # radius 半径
    # algorithm:{'auto', 'ball_tree', 'kd_tree', 'brute'} 指定索引方法
    # kd_tree 针对方格子进行的索引  ball_tree 针对球体进行的索引  brute 遍历
    # p 就是闵可夫斯基距离公式中的 p 默认为2 
    # n_jobs 并行计算的单元数量
    knn_clf = KNeighborsClassifier(n_neighbors=3)
    knn_clf.fit(X_train, Y_train)
        
    # 利用训练集进行预测
    print("Train")
    Y_pred = knn_clf.predict(X_train)
    print("ACC:", accuracy_score(Y_train, Y_pred))
    print("REC:", recall_score(Y_train, Y_pred))
    print("F-Score:", f1_score(Y_train, Y_pred))
    
    print("Validation")
    # 预测 验证集 得到一个预测值
    Y_pred = knn_clf.predict(X_validation)
    # 拿预测值与实际值进行比较
    print("ACC:", accuracy_score(Y_validation, Y_pred))
    print("REC:", recall_score(Y_validation, Y_pred))
    print("F-Score:", f1_score(Y_validation, Y_pred))
    
    # 利用测试集进行对比
    print("Test")
    Y_pred = knn_clf.predict(X_test)
    print("ACC_test:", accuracy_score(Y_test, Y_pred))
    print("REC_test:", recall_score(Y_test, Y_pred))
    print("F-Score_test:", f1_score(Y_test, Y_pred))
    
    # 将模型进行存储
    from sklearn.externals import joblib
    # 保存模型
    # joblib.dump(knn_clf, "knn_clf")
    # 使用模型
    knn_clf = joblib.load("knn_clf")
    print("Test_2")
    Y_pred = knn_clf.predict(X_test)
    print("ACC_test:", accuracy_score(Y_test, Y_pred))
    print("REC_test:", recall_score(Y_test, Y_pred))
    print("F-Score_test:", f1_score(Y_test, Y_pred))
    
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    hr_modeling(features, label)
if __name__=="__main__":
    main()

Train
ACC: 0.9742193577064118
REC: 0.959794296400187
F-Score: 0.9465191332411249
Validation
ACC: 0.9546666666666667
REC: 0.9364406779661016
F-Score: 0.9069767441860463
Test
ACC_test: 0.959
REC_test: 0.9281767955801105
F-Score_test: 0.9161554192229038
Test_2
ACC_test: 0.9703333333333334
REC_test: 0.9502762430939227
F-Score_test: 0.9392491467576791


### 朴素贝叶斯 

In [26]:
# 概率：P(A)
# 条件概率：P(A|B) 条件B发生的条件下，条件A发生的概率
# 联合概率：P(A, B) 条件A,B同时发生的概率 
# P(A, B) = P(A|B)P(B) = P(B|A)P(A)
# 全概率公式：B的概率等于：A事件发生的各种情况下，B事件发生的概率的和
# 拉普拉斯平滑：若存在0值，就将所有的数据都加上1，可以规避样本数量等于0的情况

In [28]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)
        
    
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    # print(len(X_train), len(X_validation), len(X_test))
    
    # 衡量指标
    # accuracy_score 准确率
    # recall_score 召回率
    # f1_score F值
    from sklearn.metrics import accuracy_score, recall_score, f1_score
    # KNN
    # NearestNeighbors 此方法可以直接获得一个点附近最近的几个点
    from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
    # 朴素贝叶斯 （适用与离散型的数值）
    # GaussianNB 高斯朴素贝叶斯 （特征是高斯分布的）
    # BernoulliNB伯努利朴素贝叶斯 （数值是离散的，二值，0和1）若是连续值，也需要先将数值离散化
    from sklearn.naive_bayes import GaussianNB, BernoulliNB
    models = []
    # n_neigbors 获取最近的几个点
    # radius 半径
    # algorithm:{'auto', 'ball_tree', 'kd_tree', 'brute'} 指定索引方法
    # kd_tree 针对方格子进行的索引  ball_tree 针对球体进行的索引  brute 遍历
    # p 就是闵可夫斯基距离公式中的 p 默认为2 
    # n_jobs 并行计算的单元数量
    models.append(("KNN", KNeighborsClassifier(n_neighbors=3)))
    # 高斯贝叶斯
    models.append(("GaussianNB:", GaussianNB()))
    # 伯努利朴素贝叶斯
    models.append(("BernoulliNB:", BernoulliNB()))
    
    for clf_name, clf in models:
        # 先进行拟合 训练
        clf.fit(X_train, Y_train)
        # 进行预测和评价 【训练集，验证集，测试集】
        xy_lst = [(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)]
        for i in range(len(xy_lst)):
            # X_part
            X_part = xy_lst[i][0]
            # y_part
            Y_part = xy_lst[i][1]
            # 预测值
            Y_pred = clf.predict(X_part)
            # 0训练集 1验证集 2测试集
            print(i)
            print(clf_name, "-ACC:", accuracy_score(Y_part,  Y_pred))
            print(clf_name, "-REC:", recall_score(Y_part,  Y_pred))
            print(clf_name, "-F-Score:", f1_score(Y_part,  Y_pred))
    
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    hr_modeling(features, label)
if __name__=="__main__":
    main()

0
KNN -ACC: 0.974886098455384
KNN -REC: 0.9619489559164733
KNN -F-Score: 0.9483074107959744
1
KNN -ACC: 0.9543333333333334
KNN -REC: 0.9188034188034188
KNN -F-Score: 0.9039943938332164
2
KNN -ACC: 0.95
KNN -REC: 0.927170868347339
KNN -F-Score: 0.8982360922659431
0
GaussianNB: -ACC: 0.796199577730859
GaussianNB: -REC: 0.7461716937354989
GaussianNB: -F-Score: 0.6368316831683168
1
GaussianNB: -ACC: 0.7953333333333333
GaussianNB: -REC: 0.7393162393162394
GaussianNB: -F-Score: 0.6283292978208234
2
GaussianNB: -ACC: 0.8003333333333333
GaussianNB: -REC: 0.7310924369747899
GaussianNB: -F-Score: 0.6354230066950699
0
BernoulliNB: -ACC: 0.8413157017446383
BernoulliNB: -REC: 0.46728538283062643
BernoulliNB: -F-Score: 0.5851249273678093
1
BernoulliNB: -ACC: 0.845
BernoulliNB: -REC: 0.47150997150997154
BernoulliNB: -F-Score: 0.5874001774622893
2
BernoulliNB: -ACC: 0.8386666666666667
BernoulliNB: -REC: 0.469187675070028
BernoulliNB: -F-Score: 0.5805892547660312


### 生成模型与判别模型

In [ ]:
# 生成模型（对数据要求高，速度快一些）：通过求输入与输出的联合概率分布，再求解类别归类的概率。比如朴素贝叶斯模型（因为其分子是一个联合概率分布）
# 判别模型（数据要求不高，速度慢一些，使用的范围更广）：不通过联合概率分布，直接就可以获得输出对应最大分类的概率。比如KNN